In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import dlib
from PIL import Image

In [2]:
import torch
from models import ResNet50, mobilenet
from torchvision import transforms
from torch.utils.data import DataLoader
from dataset.RGBD_transforms import Resize

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
pretrained_model_path = "3dface_models/logs_mobileNet_v2_12-15.14-31/3dface-model.pkl"

In [5]:
input_channels = 4
num_of_classes = 1200

model = mobilenet(input_channels, num_of_classes, pretrained=False)
# model = ResNet50(input_channels, num_of_classes, pretrained=False)
model.load_state_dict(torch.load(pretrained_model_path, map_location=device))
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()

model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [6]:
train_transform = transforms.Compose([
                    Resize(224),
                    transforms.ToTensor(),
                ])

In [7]:
def predict(image):
    tensor_RGBD = train_transform(image)
    tensor_RGBD= tensor_RGBD.to(device)  
    predictions = model(tensor_RGBD[None, ...])
    return predictions

In [8]:
# load face detection model
detector = dlib.get_frontal_face_detector()

# Create a pipeline
pipeline = rs.pipeline()

# Create a config and configure the pipeline to stream
# different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)

In [9]:
list_name = ['Chitsanupong','Chanawat']

In [10]:
face_saved = [[0.3265908658504486, 0.9663196802139282, 0.19752678275108337, -0.7273368835449219, 0.2435065060853958, 0.6268201470375061, 0.11746812611818314, -0.0973156988620758, -0.5596145391464233, -0.222126305103302, -0.4125766158103943, 0.02912791445851326, 0.6326131820678711, 0.13544756174087524, 0.13261650502681732, -0.050569869577884674, -0.4141579568386078, -0.06164902448654175, -0.03666974604129791, -1.0598466396331787, -0.546840488910675, 0.8030062317848206, -0.35394397377967834, 0.9023665189743042, -0.7656869888305664, 0.42391714453697205, -0.7866078615188599, -0.4073209762573242, 1.0106585025787354, 0.6063457131385803, -0.7716904878616333, -0.04039914533495903, -0.21211016178131104, -0.5883469581604004, 0.40673381090164185, 0.34543681144714355, 0.2510216236114502, -0.6846190094947815, 0.2649911344051361, 0.5038203001022339, 0.4591865539550781, -0.4709819257259369, -0.9750238656997681, 0.13046668469905853, -0.20502328872680664, 1.3292120695114136, -1.0545107126235962, -0.40657031536102295, -0.3981633484363556, 0.037247851490974426, -0.6238422393798828, -0.2552690804004669, -0.7260084748268127, -0.7809687852859497, -0.6278068423271179, -0.18796414136886597, -0.8973796963691711, -0.9540467262268066, -0.9525635242462158, -0.8162676095962524, -0.17995786666870117, -1.6448110342025757, -0.5656854510307312, -0.13232553005218506, 0.07779216021299362, 1.9445207118988037, 0.3474588692188263, -0.16390904784202576, -0.8968552350997925, -0.15885765850543976, -0.08248640596866608, -0.17877918481826782, -0.746096670627594, 0.03313032537698746, 0.4974055588245392, -0.9421638250350952, -0.9504343271255493, -0.19035866856575012, -0.8084940910339355, -0.13948816061019897, -0.3933510184288025, -0.7384290099143982, 0.3205815255641937, 0.6827161312103271, -0.16825038194656372, -0.056170105934143066, 1.1333136558532715, -0.2463001161813736, 0.2933676540851593, 0.2699456512928009, -0.07720814645290375, -1.336627721786499, 0.5134666562080383, 0.4340457022190094, -0.015641234815120697, -0.5068272352218628, -0.19992876052856445, 0.07481876015663147, -0.10454340279102325, -0.0786437913775444, 0.10107439756393433, -0.006781076546758413, 0.2028922438621521, 1.590313196182251, 1.003220558166504, 0.010572904720902443, 0.9447008967399597, 0.41193243861198425, -0.2111615389585495, -0.36355629563331604, 0.6729751825332642, -0.030757077038288116, 0.22316470742225647, 0.5116135478019714, 0.21924492716789246, -0.5110774040222168, -0.12425850331783295, -0.15891435742378235, 1.3606599569320679, 1.2203940153121948, 0.6266396045684814, 0.15589380264282227, 1.044180154800415, 0.7577251195907593, 1.1609059572219849, 0.23537588119506836, 0.5884263515472412, 0.7969503402709961, 0.01867913268506527, 0.47053712606430054, 0.2904398739337921, 0.15910999476909637, -0.06260660290718079, 0.5070279836654663, -0.1819070428609848, 0.5248931050300598, 0.9231305122375488, -0.0823921412229538, -0.5837069749832153, -0.6483725905418396, 0.018262214958667755, 0.23020678758621216, 0.6818027496337891, -0.27570396661758423, -0.4904356002807617, -0.7828649282455444, -0.2951645851135254, -0.4165645241737366, 0.3987986743450165, -0.3236505091190338, -0.40983614325523376, -0.47355493903160095, 0.06789965182542801, -1.3728790283203125, 0.052363570779561996, 0.22316360473632812, -0.8256179094314575, 0.5778890252113342, 0.6776725649833679, -0.35266774892807007, -0.12172664701938629, -0.07677628099918365, 0.20588648319244385, 0.21121735870838165, 0.1488845944404602, -0.6241812705993652, -0.9362719655036926, -0.7434842586517334, -0.8510803580284119, -0.6058691143989563, -0.8931018710136414, -0.06255722045898438, 0.45797696709632874, -0.25286969542503357, 1.1801995038986206, -0.6919234991073608, 0.6973740458488464, -0.8436023592948914, 0.20386111736297607, 0.030214421451091766, 0.21246683597564697, 0.4565143883228302, 0.09870393574237823, -0.3896046578884125, -0.1391991227865219, 1.0299688577651978, -0.35969454050064087, 0.2636483907699585, -0.2057924121618271, -0.044765982776880264, 1.0870888233184814, 0.6898924708366394, 0.03694665804505348, 0.15943913161754608, 0.37140217423439026, -0.992499828338623, -0.16386957466602325, 0.6620315909385681, -1.0647602081298828, 1.1650813817977905, -0.41020649671554565, 0.05731917917728424, -0.10766260325908661, 0.37051209807395935, -0.2449914962053299, -0.7328956723213196, 1.6773631572723389, -0.048704102635383606, -0.4379676878452301, -0.6306925415992737, -0.6281818151473999, -0.4366283118724823, -0.3162235915660858, 0.0886303186416626, -0.8239018321037292, -0.20457834005355835, -1.0036332607269287, -0.05950689688324928, -0.2883948087692261, 0.3119431436061859, -0.32819804549217224, -0.80531907081604, -1.0986324548721313, 1.2785422801971436, -0.7314960360527039, -0.22200119495391846, -0.8116170763969421, 0.6720230579376221, 0.10448014736175537, -0.11883597075939178, -0.2504238784313202, 0.06240561604499817, 0.8550761938095093, -0.13234107196331024, -0.5177416205406189, -0.44484996795654297, 0.551173746585846, 0.07630722224712372, -0.5360836386680603, 0.860420823097229, 0.19057141244411469, 0.435982882976532, 0.41610363125801086, -0.05044649541378021, 0.6321434378623962, 1.0262633562088013, 0.3061995506286621, 0.5629981160163879, 0.03252241387963295, 0.48961058259010315, 1.1481685638427734, -0.09815312176942825, 0.3514019846916199, 1.2506169080734253, 1.0297771692276, 0.11768731474876404, -0.38955458998680115, 0.4117160439491272, 0.013282778672873974, -0.4582761526107788, -0.3578226864337921, 0.8159677386283875, 0.3590136468410492, 0.24110636115074158, -0.036847859621047974, -0.04944666102528572, -0.05022429674863815, 0.7247688174247742, 0.09014971554279327, 1.0160844326019287, -0.2174433171749115, 0.11145269125699997, 0.7259492874145508, 1.494983434677124, 0.045481011271476746, -0.6933583617210388, 0.20643199980258942, 0.9852554202079773, 0.49367690086364746, 1.44935142993927, -0.23976807296276093, 0.5696951746940613, -0.047073572874069214, 0.5212079882621765, -0.45295053720474243, 0.6710503697395325, -0.4552382826805115, 0.3832940459251404, -0.44096171855926514, -0.42493733763694763, -0.6005300283432007, -0.05909645929932594, 0.4291473925113678, -0.5073951482772827, -0.5533913373947144, 0.527438223361969, 0.373915433883667, 0.2172362059354782, 0.31680381298065186, -0.5485666394233704, 0.1734558343887329, -0.19698481261730194, -0.2227427214384079, -0.6727240085601807, -0.4879485070705414, -0.44422173500061035, 0.8544945120811462, 0.13611151278018951, 0.638765811920166, -0.1375487744808197, 0.1815134733915329, -0.9701848030090332, 1.0968577861785889, 0.22783130407333374, 0.5513332486152649, -0.09697897732257843, 0.5436053276062012, 0.9462160468101501, -0.18805450201034546, -0.2944042384624481, -0.935966432094574, -0.2114698886871338, -0.3332015573978424, 0.3474651277065277, -0.49854621291160583, -0.19719909131526947, -0.619455099105835, 0.7642159461975098, 0.07518868893384933, -0.17626483738422394, -0.4233214259147644, -0.4190058410167694, -0.10030528157949448, -0.19984617829322815, -0.2395908534526825, -0.7379317283630371, 0.3227563500404358, 0.5560519099235535, -0.3010294735431671, -0.6018596887588501, 0.07351744920015335, -0.32060620188713074, 1.2375977039337158, 0.08777937293052673, -0.6650472283363342, -1.0407087802886963, -0.4037550687789917, -1.5938808917999268, 0.17746838927268982, -0.3422998785972595, 0.41247984766960144, 0.1776692271232605, -0.6202433705329895, 0.31484249234199524, 0.6979768872261047, 0.0014717174926772714, 0.5617831945419312, 0.3785354793071747, -0.6063401699066162, -0.7862751483917236, -0.21645955741405487, -0.3751503527164459, 0.32703158259391785, -0.24238711595535278, 0.0679280161857605, -0.532757580280304, -0.31996670365333557, -0.23570580780506134, 0.07893056422472, -0.48522889614105225, 0.16108107566833496, 0.3943457305431366, 0.053813569247722626, -0.8865105509757996, -0.23278799653053284, -0.6253155469894409, -0.522282063961029, 0.4688255190849304, 0.9743065237998962, -0.2860534191131592, 0.1501714289188385, -0.8095687031745911, 0.9368297457695007, 0.22519654035568237, 0.34426647424697876, -0.6895551085472107, 0.2359851598739624, -0.11391957849264145, -0.5249914526939392, 0.2915407717227936, 0.20812773704528809, 0.14953801035881042, -0.4577277600765228, -0.043341897428035736, 0.5332927107810974, 0.41215527057647705, -0.5151079893112183, 0.1555991768836975, -0.30171188712120056, 0.4226624667644501, 0.20824560523033142, 0.33202576637268066, -1.3519070148468018, 0.11926975846290588, -0.29880863428115845, -1.129560112953186, -0.9220903515815735, 0.42392826080322266, -0.42273446917533875, 0.10180047154426575, 0.7054802775382996, -0.5627038478851318, 0.03142382949590683, -0.2740556299686432, 0.8793430328369141, 0.47046414017677307, -0.7999160289764404, 0.24059487879276276, -1.123880386352539, -0.6264082193374634, -0.1723608523607254, -0.09417534619569778, -0.3118921220302582, 0.7267111539840698, 0.4577939510345459, 0.8114519715309143, 0.7693770527839661, 0.03434174507856369, -0.5971795320510864, 0.8801414966583252, -1.6319915056228638, 0.7676218152046204, 0.2838535010814667, 0.6285361051559448, 1.0402778387069702, -0.8420023918151855, 0.5804325938224792, 0.9955348968505859, -0.16064052283763885, -0.22345952689647675, -0.3081066310405731, -0.10747203230857849, -0.31428706645965576, -0.5128015875816345, -0.839076042175293, -0.28566256165504456, -0.1190502941608429, 0.1018715500831604, -0.11616947501897812, -0.12752394378185272, 0.4981273412704468, -0.22918017208576202, 0.13093416392803192, -0.17173431813716888, -0.5900571942329407, 0.03448881581425667, -0.6862615346908569, -0.04400074481964111, -0.11014431715011597, -0.18411099910736084, 0.25249049067497253, -0.12580813467502594, -0.4197908341884613, -0.14043603837490082, -0.16285036504268646, -1.179623007774353, 1.087424874305725, -0.8119133710861206, 0.09049978107213974, -0.08931802213191986, -0.6266268491744995, -0.35478177666664124, -0.898009181022644, -0.8530170321464539, -0.8706064820289612, 0.10594570636749268, 0.6447597742080688, -1.2224736213684082, 0.24368491768836975, -0.815659761428833, 0.41678863763809204, 0.8026073575019836, 0.591189980506897, 1.2905391454696655, -0.28716495633125305, -0.2060033082962036, 0.6853958964347839, 0.8126903772354126, 0.07724330574274063, -0.9017974138259888, -0.35263022780418396, -0.002226768061518669, -0.18227112293243408, 0.044311583042144775, -1.0119678974151611, 0.010595031082630157, -0.5578727126121521, 0.5612029433250427, 0.9024782180786133, 0.6116438508033752, 0.2038806676864624, 0.4934888780117035, -0.044950295239686966, 0.816392719745636, -0.1693161427974701, 0.1327729970216751, 0.0973559021949768, 0.31439128518104553, -0.21009470522403717, -0.0678553506731987, -0.7334551215171814, -0.5630636215209961, 0.5007363557815552, 0.30429011583328247, 0.4934878647327423, -0.06613639742136002, 0.27506500482559204, -0.25353333353996277, -0.08119119703769684, 0.9199264049530029, 0.18812111020088196, 0.4745107889175415, -0.6331964731216431, 0.4028838574886322, 0.019382890313863754, -0.3352503180503845, -0.05938529968261719, 0.27690908312797546, 1.1686710119247437, 0.5379411578178406, 0.1589343249797821, -0.4898228943347931, -0.9011781215667725, -0.127182736992836, -0.4528777301311493, -0.6794555187225342, -0.10572508722543716, -0.5454891920089722, -1.297063946723938, -0.5034949779510498, -1.1641004085540771, -0.9228906035423279, 0.7045322060585022, -0.7010523676872253, -0.4565235674381256, 0.42053958773612976, -0.4534597396850586, -1.1420042514801025, 0.7364757657051086, 0.38978368043899536, 0.8989065289497375, -0.5676190853118896, 0.8691699504852295, 0.21652860939502716, 0.23645086586475372, -0.6583713889122009, -0.07724881917238235, -0.6415187120437622, -0.4283078908920288, -0.306561142206192, 0.0045714955776929855, 0.5988988876342773, 0.5277926921844482, -0.9359014630317688, 0.08569332212209702, 1.0873442888259888, -1.0207796096801758, -0.18799230456352234, -0.5917502641677856, -0.9878190159797668, -0.521845281124115, -0.7784707546234131, 0.25332510471343994, -0.4506306052207947, -1.343384027481079, -0.5027909874916077, -0.5589869618415833, 0.05704588070511818, -0.8140098452568054, -0.06584920734167099, 0.8870906829833984, -0.04269219934940338, 0.8834566473960876, -0.4340750575065613, 0.09175741672515869, -0.693757176399231, 0.7176143527030945, -0.359622985124588, -0.2182036191225052, -0.6005933880805969, 0.21948568522930145, 1.108527421951294, 0.7743893265724182, 0.8964366912841797, 0.47877994179725647, 0.49963539838790894, 0.6898845434188843, 0.01600363291800022, -0.2086315006017685, 0.07048139721155167, 0.2213815450668335, 0.30265820026397705, -0.43273302912712097, -0.13958224654197693, 0.4001259207725525, -0.8402076959609985, -0.29518768191337585, 0.6166749000549316, 0.6527730226516724, 1.1558095216751099, 0.4000428318977356, 0.7491798400878906, -0.19843928515911102, -0.2497093379497528, -0.06043841317296028, -0.5481532216072083, 0.9474663138389587, 0.5836616158485413, -0.023192336782813072, 0.5295155644416809, -1.1185274124145508, 0.07706981897354126, -0.6994842886924744, -0.4457550048828125, -0.7657954692840576, 0.1687847077846527, 0.9952303767204285, -0.17790581285953522, -0.9420014023780823, 0.8959506154060364, 1.1028307676315308, -0.3921116888523102, -0.706089198589325, -0.2112564742565155, 0.2658292353153229, 0.5182909369468689, 0.38547131419181824, -0.3764803409576416, -0.2841535806655884, 0.028089333325624466, -0.35290342569351196, -0.44210463762283325, 0.7630556225776672, -0.31045404076576233, -0.46460092067718506, -1.1711164712905884, 0.3821270167827606, 0.43630799651145935, -0.8427063226699829, -0.07511167973279953, 0.1933988481760025, 0.10557020455598831, 0.05663791298866272, 1.0056824684143066, 0.10287346690893173, 0.9277870655059814, 0.5798373222351074, -0.585409939289093, 0.8219668865203857, 1.4797924757003784, 0.20010678470134735, -0.5558710098266602, 0.685249924659729, 0.4144761264324188, 0.2028540074825287, -0.6328858137130737, -0.09050977230072021, 0.06350165605545044, 0.35114938020706177, -0.447942316532135, -0.8435159921646118, -0.1716458648443222, 2.251741409301758, 0.641746997833252, -0.7443767786026001, 0.6647873520851135, -0.03580667823553085, -0.0359041802585125, -0.5269839763641357, -0.42045798897743225, -0.7573626041412354, 0.7070666551589966, -0.2800295054912567, 0.12068460136651993, -0.39863187074661255, 0.6292238831520081, 0.8125719428062439, 0.36111193895339966, 1.042587399482727, 0.13265135884284973, -0.7025339007377625, 0.43234577775001526, -0.7964531779289246, 0.15034180879592896, -0.1904781013727188, -0.5317609310150146, 0.2598060369491577, -1.312295913696289, 0.3148297369480133, 0.3431156575679779, -0.6879947781562805, 0.5635133385658264, 0.6846564412117004, -0.9384422898292542, 0.0548066720366478, -0.40835076570510864, -1.1932506561279297, -0.2474578619003296, -0.17444725334644318, 0.4050615429878235, 1.2297533750534058, -0.06883475929498672, -0.040159642696380615, -1.059995412826538, -0.815079391002655, -0.03637097403407097, -0.04590555280447006, -0.0250085461884737, 0.3871902525424957, -0.22684846818447113, -0.7737247943878174, 0.20890796184539795, -1.0969767570495605, -0.9424766898155212, -0.1897091567516327, -0.6454220414161682, -1.193071722984314, 0.39397433400154114, 0.5900701284408569, -0.4726271331310272, 0.14916133880615234, -0.12337826937437057, -0.23887421190738678, -0.9424895644187927, 0.38439664244651794, -1.1360293626785278, -0.3876742720603943, -0.47053223848342896, -1.6172409057617188, -0.7890008091926575, -0.6440997123718262, -0.04557451978325844, 0.5254284739494324, -1.0581022500991821, -0.5651383399963379, -0.7837293744087219, 1.5365222692489624, -0.07465100288391113, -0.48999908566474915, 1.2428748607635498, 0.15641623735427856, 0.6737266778945923, 0.5943726301193237, 0.7873257994651794, 0.25169140100479126, -0.22917218506336212, 0.06141456961631775, -0.0295412577688694, -0.05230236053466797, 1.2810391187667847, -0.10282019525766373, 0.4319704473018646, -0.07428431510925293, -0.1372937709093094, -0.14429132640361786, -0.03299062326550484, -0.87947678565979, -0.46873077750205994, 0.44720369577407837, -0.24583794176578522, 0.7197783589363098, -0.868880569934845, -0.7247028946876526, 0.03503699228167534, -0.41494256258010864, -0.3707975149154663, 0.01816413551568985, 0.6635608673095703, -0.10228133946657181, -1.1460853815078735, 0.7520555257797241, -0.5831712484359741, -0.11952073872089386, -1.6156326532363892, -0.8061593770980835, 0.2705768048763275, 0.2054305076599121, 0.6572320461273193, -0.5086178183555603, 0.9113442897796631, 0.39433562755584717, 0.6953030824661255, 0.6491714715957642, -0.4032796025276184, -0.406659871339798, 0.8442323803901672, 0.79067063331604, -0.8974273800849915, 0.29801464080810547, -0.42082729935646057, -0.6972907781600952, 1.2792600393295288, -0.37733685970306396, 0.5921991467475891, 0.7574063539505005, -0.05447155982255936, -0.12679453194141388, -0.8656377196311951, -0.4507383704185486, 0.44218990206718445, -0.4640783667564392, -0.0638532042503357, 0.13724012672901154, -0.8537749648094177, 0.23646023869514465, 0.5134904980659485, 0.9824472069740295, 0.3203236162662506, 0.4341288208961487, 0.1914365142583847, 0.006904687732458115, -0.9830248355865479, 0.0913519412279129, -0.35418298840522766, -0.44097208976745605, -0.509337306022644, 0.3315100371837616, 0.8708645701408386, 1.5622172355651855, 1.583916425704956, -0.17935271561145782, 0.7892094254493713, 1.023181676864624, 1.1712603569030762, 0.1340201497077942, -0.19808977842330933, -0.18026995658874512, -0.18614009022712708, -0.7942237854003906, 0.07122265547513962, -1.0291030406951904, -0.13576216995716095, -0.046891190111637115, -0.6818186044692993, -0.5511893630027771, -0.4504414498806, -0.001980964094400406, -0.6643872857093811, -0.006809592247009277, 0.3435819447040558, 0.22173109650611877, 0.6839987635612488, -0.09158533066511154, -0.27822354435920715, 0.36938390135765076, 0.6644407510757446, 0.03092121332883835, 0.21136660873889923, -0.5364733338356018, 1.3087061643600464, -0.3974744379520416, 0.317402720451355, -0.40510284900665283, -0.17235809564590454, -0.03647647053003311, -0.08617866039276123, 1.1989222764968872, -0.7443608045578003, -0.3432367742061615, -0.8504669070243835, 1.112200140953064, -0.7229092121124268, -0.4016996920108795, -1.0791184902191162, 1.4209226369857788, 0.34234002232551575, -0.5980446338653564, 0.008104270324110985, -0.49411624670028687, 0.1655476838350296, -0.7339323163032532, 0.03896884620189667, 1.131009817123413, 0.3790097236633301, -0.7935096025466919, -0.4426330626010895, 1.0552433729171753, -0.15042002499103546, 0.12193703651428223, 0.2745046615600586, -0.1468261182308197, 1.19879150390625, 0.29893288016319275, 0.6942442655563354, 0.7389863729476929, -0.8532969951629639, 0.059180233627557755, 0.4339969754219055, 1.0022555589675903, 0.05109124630689621, 0.06449245661497116, -0.16626299917697906, 1.4872426986694336, 0.07987770438194275, 0.1616494059562683, 1.1005374193191528, -0.6759453415870667, 0.41304877400398254, -0.19998866319656372, -0.5250051021575928, -0.40163639187812805, -0.7825787663459778, 0.15021884441375732, -0.7682746052742004, -0.36243635416030884, -0.22164982557296753, 1.0072938203811646, -0.8723028302192688, 0.054869189858436584, -0.755365252494812, 1.046372652053833, 0.160615935921669, -0.34190070629119873, 0.46906986832618713, -1.1841081380844116, 0.440580815076828, -0.5218245387077332, -0.5173554420471191, -0.31250786781311035, -0.6365705132484436, 1.6447805166244507, -0.061246417462825775, 0.17719335854053497, -0.5700798034667969, -0.4854736626148224, -0.5195701122283936, 0.20533151924610138, -0.3574471175670624, -0.1883162409067154, 0.1946781724691391, -0.039083097130060196, 0.3017643392086029, -1.1830470561981201, -0.004426812753081322, 0.10160300135612488, -0.7593074440956116, 1.0955309867858887, 0.0362810455262661, -0.6394719481468201, 0.7535256743431091, -0.875868558883667, 1.1467174291610718, 0.9236763119697571, -0.07899119704961777, 0.349841445684433, -0.4592615067958832, -0.9718542695045471, -0.43589940667152405, 1.314436435699463, 0.3870559632778168, 0.46030905842781067, 1.9454187154769897, 0.5293425917625427, 0.7765278816223145, 0.2550632655620575, -1.107783555984497, 0.38530153036117554, -0.38552382588386536, -0.1747615933418274, -0.9377865791320801, -1.190266489982605, 0.49686795473098755, 0.10810849070549011, -0.029055804014205933, 0.6867246627807617, 0.4565960168838501, 2.15342378616333, -0.1611892282962799, 0.020417585968971252, 0.10073447227478027, -0.7808613181114197, 0.5160955786705017, 0.24859586358070374, -0.48486167192459106, -0.46172216534614563, 0.6144652962684631, 0.6131971478462219, 0.3971147835254669, -0.17647013068199158, 0.1769285947084427, 0.4392203986644745, -0.36321723461151123, -0.9074954986572266, -0.037339963018894196, -1.1256052255630493, 0.1850663125514984, -0.1841553896665573, -0.22638638317584991, -0.5990951061248779, -0.4895019829273224, 0.8804501891136169, -0.32897359132766724, 0.2508847117424011, -0.5384612083435059, 0.7260950803756714, 0.4206644296646118, 1.0313045978546143, -0.2874186635017395, -0.5777922868728638, -0.08516890555620193, -0.22072024643421173, -0.01722279191017151, -0.7077199220657349, -0.11551392078399658, -0.6310239434242249, -1.0450124740600586, -0.8202476501464844, 1.3672821521759033, -0.21739134192466736, 0.49018433690071106, -0.9173073768615723, 0.07524331659078598, 0.8953216075897217, 1.7140614986419678, -0.5970014333724976, 0.5311110615730286, 0.02851482667028904, 0.25283941626548767, 0.2574811577796936, 0.3492140471935272, -0.8028823137283325, 0.31327134370803833, -0.706660270690918, -0.06437935680150986, -0.14788107573986053, 0.8841423988342285, -0.25083765387535095, 0.7292925119400024, -0.22737300395965576, 0.2947385311126709, 0.7705922722816467, 0.2810937166213989, 1.6037907600402832, 0.2852857708930969, 1.2631608247756958, 0.27832522988319397, 0.7578630447387695, 0.9559889435768127, 0.5625356435775757, -0.3596653342247009, -0.26651111245155334, 0.05314142256975174, 0.510022759437561, 0.23478692770004272, 0.4075823426246643, 0.6645320057868958, -0.1596696376800537, 0.34437739849090576, 0.40647974610328674, -0.04540398716926575, 0.3913669288158417, 0.5612400770187378, 0.45650196075439453, -0.11751740425825119, -0.526205837726593, -0.48373186588287354, 0.5721456408500671, -0.9393841028213501, -0.6438359022140503, 0.07238637655973434, -0.534289538860321, -0.6892038583755493, 0.6861647367477417, 0.39564886689186096, 0.27423015236854553, 0.3362707197666168, 0.06709413975477219, -0.16337433457374573, 0.5126574039459229, -0.4782887399196625, 0.7374221682548523, -0.1610313206911087, 0.5921367406845093, 0.6130549907684326, 0.9051740765571594, 0.41611313819885254, 1.030068039894104, 0.28439927101135254, -0.49596279859542847, 0.1792144924402237, 0.7019046545028687, -0.45005080103874207, 0.6036679148674011, -0.1308056265115738, 0.3188450336456299, 0.9018511772155762, -0.5261672139167786, 0.4702696204185486, -0.5057162642478943, -0.17280995845794678, 0.18016165494918823, 0.8380932211875916, 0.8102703094482422, -0.843310534954071, -0.3415754437446594, -0.08112704753875732, -0.48786401748657227, 0.49102452397346497, -0.18894898891448975, -0.8512384295463562, -0.6260614395141602, 0.6026111841201782, -0.45560571551322937, -0.005838260054588318, 1.5847082138061523, -0.3644528090953827, -0.2072557806968689, 0.38981693983078003, -0.7776739597320557, 0.34023502469062805, 1.0026304721832275, 0.5769307613372803, 0.15154515206813812, 1.0880568027496338, -0.8284761905670166, -0.19673621654510498, 0.8096423149108887, -0.032395124435424805, 0.10995863378047943, 0.7933183908462524, -0.08242421597242355, -0.12147609144449234, 0.01938459649682045, -0.16819874942302704, -0.02103971503674984, 0.21679578721523285, -0.7510730028152466, 0.0573272742331028, 0.11651355773210526, 0.09162241220474243, -0.014118382707238197, 0.36814460158348083, 0.4764159023761749, -0.08723744750022888, -0.07682512700557709, -0.3596028685569763, -0.3552425503730774, 0.8630629181861877, -1.0363497734069824, -1.2495933771133423, -0.39686498045921326, 0.4055994749069214, 0.0914142057299614, 0.4450026750564575, -0.374072402715683, -1.0496524572372437, 1.2539334297180176, 0.588364839553833, -0.7769501805305481, -0.6789095401763916, 0.23084911704063416, -1.055957555770874, -0.4621366262435913, 0.5516307353973389, 0.8805239200592041, 1.3513015508651733, 0.274559885263443, -0.8386995792388916, -0.7061591744422913, -0.20932546257972717, 0.413338303565979, 0.45506757497787476, -0.06499240547418594, -0.21525408327579498, -0.9151264429092407, 1.1237162351608276, -0.362701416015625, -0.0777357742190361, -0.7193152904510498, 0.29686439037323, 0.2535307705402374, -0.11278823763132095, 0.5603452324867249, 0.2963384985923767, -0.4330998361110687, -0.6347213983535767, -0.0006682173116132617, 0.4916640818119049, 0.24757997691631317, -0.7363834381103516, -0.413979172706604, 0.0677504613995552, 0.3403444290161133, -0.8685122728347778, -1.2331641912460327, -0.3325969874858856, -0.5317203402519226, -0.5777991414070129, -0.2933007478713989, -0.4072439670562744],
            [0.29449713230133057, 0.9643260836601257, 0.24630725383758545, -0.7107711434364319, 0.3034304976463318, 0.63460373878479, 0.09964235872030258, -0.08441520482301712, -0.554806649684906, -0.23983585834503174, -0.41942405700683594, 0.07866368442773819, 0.547687292098999, 0.1511562019586563, 0.14872391521930695, -0.07922258973121643, -0.42412254214286804, -0.011940808966755867, -0.027989087626338005, -1.050570011138916, -0.5254260897636414, 0.7927265167236328, -0.3193456828594208, 0.9329642057418823, -0.7671773433685303, 0.4535689949989319, -0.7792508006095886, -0.5313019752502441, 1.1053709983825684, 0.6387178897857666, -0.7840754389762878, -0.01402595266699791, -0.20516635477542877, -0.5117192268371582, 0.3824719786643982, 0.3282355070114136, 0.2588621973991394, -0.6797136068344116, 0.3370346128940582, 0.5067481398582458, 0.4232361316680908, -0.49539467692375183, -0.9789421558380127, 0.12105667591094971, -0.12885542213916779, 1.358392357826233, -1.027345061302185, -0.425162672996521, -0.40900638699531555, 0.004951700568199158, -0.6329296231269836, -0.30054566264152527, -0.7244099378585815, -0.7956465482711792, -0.6372782588005066, -0.18891862034797668, -0.9077666401863098, -0.9067165851593018, -0.9960734248161316, -0.7592503428459167, -0.1873820573091507, -1.6005256175994873, -0.49885842204093933, -0.21249940991401672, 0.06807497888803482, 1.9715124368667603, 0.2801873981952667, -0.14987917244434357, -0.8780841827392578, -0.16479063034057617, -0.09552118182182312, -0.13657601177692413, -0.7963762879371643, -0.01859724149107933, 0.47388532757759094, -0.9235161542892456, -0.932641327381134, -0.2448108047246933, -0.7904497385025024, -0.14404723048210144, -0.3854079246520996, -0.778351902961731, 0.3385162651538849, 0.6698804497718811, -0.042531996965408325, -0.1113382950425148, 1.0163854360580444, -0.2593858540058136, 0.312090128660202, 0.30621904134750366, -0.1675795167684555, -1.3469970226287842, 0.5104671120643616, 0.4540017247200012, 0.016600120812654495, -0.5349968075752258, -0.12364721298217773, 0.05521829053759575, -0.07575656473636627, -0.14566805958747864, 0.04788828641176224, 0.019542532041668892, 0.18085819482803345, 1.583025574684143, 0.9882770776748657, -0.0685415044426918, 0.9832631349563599, 0.4973844587802887, -0.2054346352815628, -0.46053722500801086, 0.7388486862182617, -0.024222124367952347, 0.22967517375946045, 0.4409855902194977, 0.25377988815307617, -0.6132428050041199, -0.07400667667388916, -0.153511643409729, 1.368203043937683, 1.2019908428192139, 0.6223971247673035, 0.07596227526664734, 1.0204589366912842, 0.7971525192260742, 1.2170391082763672, 0.24324427545070648, 0.6178574562072754, 0.8220610618591309, 0.05621197819709778, 0.5051169395446777, 0.2926669418811798, 0.207112655043602, -0.18892420828342438, 0.499953955411911, -0.2142093926668167, 0.5811201930046082, 0.891350507736206, -0.045281827449798584, -0.5243468880653381, -0.6687938570976257, 0.06014996021986008, 0.23050951957702637, 0.6478171348571777, -0.32190239429473877, -0.5250759720802307, -0.7737507820129395, -0.25121641159057617, -0.40626007318496704, 0.3890470266342163, -0.3309975266456604, -0.4525464177131653, -0.48987576365470886, 0.06058221310377121, -1.3624399900436401, 0.1251852810382843, 0.2127174437046051, -0.7704618573188782, 0.5608373284339905, 0.748781681060791, -0.3305496573448181, -0.1701434701681137, -0.1008664071559906, 0.20076459646224976, 0.2877386510372162, 0.13875693082809448, -0.6069865822792053, -0.9207835793495178, -0.7426841855049133, -0.8795076608657837, -0.6239591240882874, -0.92934650182724, -0.04076772928237915, 0.5028305053710938, -0.24299970269203186, 1.1728461980819702, -0.6610351204872131, 0.7000805139541626, -0.8564735651016235, 0.1341654658317566, -0.006537530571222305, 0.19215160608291626, 0.4543202519416809, 0.01441098377108574, -0.40880531072616577, -0.14157511293888092, 1.007914423942566, -0.3466762900352478, 0.2789433002471924, -0.14973953366279602, 0.07301630824804306, 1.1213757991790771, 0.6257041096687317, 0.005913903005421162, 0.16221867501735687, 0.2841677963733673, -1.054666519165039, -0.11174572259187698, 0.6941945552825928, -1.052571177482605, 1.2164736986160278, -0.3177605867385864, 0.12591497600078583, -0.05910323187708855, 0.3769930601119995, -0.19682197272777557, -0.7190971970558167, 1.7059630155563354, -0.05791149288415909, -0.4557315707206726, -0.6738431453704834, -0.6188875436782837, -0.39920011162757874, -0.2542315423488617, 0.10423678159713745, -0.8589680790901184, -0.09678934514522552, -1.000598669052124, -0.09798610210418701, -0.265481561422348, 0.3427911102771759, -0.34046876430511475, -0.8149454593658447, -1.1057246923446655, 1.2497917413711548, -0.7617630958557129, -0.2580567002296448, -0.7676217555999756, 0.571520209312439, 0.07030165195465088, -0.12465143948793411, -0.22627437114715576, 0.040260180830955505, 0.7320516705513, -0.09121650457382202, -0.5115242004394531, -0.42355698347091675, 0.5397200584411621, 0.1001623198390007, -0.5425614714622498, 0.8377586603164673, 0.1920265108346939, 0.3548312187194824, 0.35060828924179077, -0.008533388376235962, 0.5745521783828735, 1.0653969049453735, 0.29175275564193726, 0.5465885996818542, 0.032066766172647476, 0.5390785336494446, 1.1320481300354004, -0.14016836881637573, 0.3728230595588684, 1.109785795211792, 0.9740367531776428, 0.18003511428833008, -0.382060706615448, 0.44094932079315186, 0.011112067848443985, -0.45716238021850586, -0.3467141091823578, 0.8513451814651489, 0.3519974946975708, 0.26283711194992065, -0.07740383595228195, -0.048815902322530746, 0.007601035758852959, 0.7603207230567932, 0.17735078930854797, 1.0344316959381104, -0.2614266872406006, 0.11202980577945709, 0.7350192070007324, 1.4929718971252441, 0.05430455505847931, -0.6883639097213745, 0.18276892602443695, 0.9521443843841553, 0.4413478970527649, 1.4750404357910156, -0.26040592789649963, 0.5453923940658569, -0.0336003340780735, 0.5980978608131409, -0.4167531728744507, 0.6524291634559631, -0.46186143159866333, 0.3523029685020447, -0.5082398056983948, -0.42988115549087524, -0.5734546780586243, -0.08010409027338028, 0.3791781961917877, -0.5233737230300903, -0.6347593665122986, 0.5002537965774536, 0.3763052225112915, 0.24610351026058197, 0.33028802275657654, -0.5246053338050842, 0.21351607143878937, -0.20049965381622314, -0.1620006114244461, -0.7270835638046265, -0.49188852310180664, -0.5014740824699402, 0.8665980696678162, 0.2069602608680725, 0.5925019383430481, -0.061180226504802704, 0.1163930669426918, -0.9083337187767029, 1.057733416557312, 0.26378127932548523, 0.6149325370788574, -0.03608541190624237, 0.5249258875846863, 0.865437388420105, -0.2727091908454895, -0.3054843246936798, -0.8698140382766724, -0.16974687576293945, -0.3169945180416107, 0.3838408291339874, -0.46508803963661194, -0.19314177334308624, -0.6227864027023315, 0.7806064486503601, 0.09927617013454437, -0.1514023393392563, -0.42049726843833923, -0.46047255396842957, -0.1189149022102356, -0.1677432358264923, -0.23677334189414978, -0.6456108093261719, 0.33696648478507996, 0.4927550256252289, -0.32243847846984863, -0.6818044781684875, 0.0900454893708229, -0.3303389251232147, 1.1601824760437012, 0.0843481719493866, -0.6824560761451721, -0.9964469075202942, -0.41123607754707336, -1.548609972000122, 0.18645158410072327, -0.3518422842025757, 0.46344757080078125, 0.15407735109329224, -0.6040229797363281, 0.256923645734787, 0.7029259204864502, 0.025734661146998405, 0.6195138096809387, 0.44384053349494934, -0.6749124526977539, -0.8065120577812195, -0.23912258446216583, -0.38780415058135986, 0.2890247404575348, -0.2439466416835785, 0.0639403760433197, -0.4919726252555847, -0.2580192983150482, -0.2913280427455902, 0.17574763298034668, -0.44389277696609497, 0.13262367248535156, 0.42381832003593445, 0.051273733377456665, -0.8873239755630493, -0.19679799675941467, -0.5641546845436096, -0.5342037081718445, 0.5466541647911072, 0.9297894239425659, -0.2939196825027466, 0.2216239720582962, -0.887237548828125, 0.9459162354469299, 0.19238387048244476, 0.3467370271682739, -0.666566789150238, 0.2527291774749756, -0.14145232737064362, -0.4946795701980591, 0.3548215925693512, 0.14523552358150482, 0.10137896239757538, -0.5138822793960571, -0.04830188304185867, 0.552179753780365, 0.4225303828716278, -0.4745182394981384, 0.15751181542873383, -0.31414783000946045, 0.3859519958496094, 0.2541593015193939, 0.3523899018764496, -1.377853274345398, 0.14736756682395935, -0.29738202691078186, -1.143791675567627, -0.9244422316551208, 0.415785014629364, -0.4055916368961334, 0.09300533682107925, 0.7882168889045715, -0.5368825793266296, -0.003018992953002453, -0.27161434292793274, 0.8459513783454895, 0.5131352543830872, -0.8002644777297974, 0.29210585355758667, -1.105981707572937, -0.6038317680358887, -0.2171364575624466, -0.03810286521911621, -0.2759142220020294, 0.7454499006271362, 0.48383432626724243, 0.812171220779419, 0.679577648639679, 0.020418142899870872, -0.5979936122894287, 0.9466514587402344, -1.6499335765838623, 0.7485501766204834, 0.2721497416496277, 0.643354594707489, 1.1180890798568726, -0.7986037731170654, 0.5977522730827332, 1.0525023937225342, -0.1531565636396408, -0.22449681162834167, -0.32171887159347534, -0.09891381859779358, -0.3410465121269226, -0.5344216227531433, -0.8279283046722412, -0.3507510721683502, -0.09457632899284363, 0.14672498404979706, -0.14410217106342316, -0.05561216175556183, 0.4789520502090454, -0.17774271965026855, 0.20084628462791443, -0.11725854128599167, -0.5166519284248352, 0.09666211903095245, -0.6789064407348633, -0.06894753873348236, -0.08366058021783829, -0.192208930850029, 0.23555496335029602, -0.09824526309967041, -0.37871500849723816, -0.09671339392662048, -0.18046891689300537, -1.1662654876708984, 1.0037763118743896, -0.8870889544487, 0.0424903929233551, -0.15434637665748596, -0.5654032826423645, -0.3238249719142914, -0.8645265102386475, -0.8380827307701111, -0.8682374954223633, 0.1300416886806488, 0.6797460317611694, -1.1859081983566284, 0.30654051899909973, -0.7876625061035156, 0.3844442367553711, 0.7634755969047546, 0.5827099680900574, 1.3383891582489014, -0.2912362813949585, -0.2778889536857605, 0.6490047574043274, 0.7744563817977905, 0.08182745426893234, -0.9162177443504333, -0.38986286520957947, -0.07995210587978363, -0.19127771258354187, -0.02082124352455139, -1.022743582725525, -0.016875430941581726, -0.5019474029541016, 0.5909208655357361, 0.9611580967903137, 0.6249735951423645, 0.20284654200077057, 0.49603599309921265, -0.03807249292731285, 0.7696810960769653, -0.10583530366420746, 0.18315325677394867, 0.1216488778591156, 0.4218961298465729, -0.13527464866638184, -0.05043017119169235, -0.7456269860267639, -0.6183847784996033, 0.5515239834785461, 0.2878115773200989, 0.48523950576782227, -0.026539424434304237, 0.3328804075717926, -0.24518203735351562, -0.12076075375080109, 0.8164631724357605, 0.14258381724357605, 0.48740071058273315, -0.6321603059768677, 0.3856685161590576, 0.03410583361983299, -0.2930532693862915, -0.039000190794467926, 0.2628399431705475, 1.1792579889297485, 0.49930039048194885, 0.11741412431001663, -0.5595778226852417, -0.8571451902389526, -0.06638016551733017, -0.41028639674186707, -0.6374249458312988, -0.10447465628385544, -0.5454941391944885, -1.281420111656189, -0.5285643935203552, -1.1374272108078003, -0.9193209409713745, 0.6892781257629395, -0.6204203963279724, -0.4485606253147125, 0.4241904616355896, -0.46366897225379944, -1.1123820543289185, 0.7352228164672852, 0.4017767608165741, 0.8092465996742249, -0.6487610936164856, 0.8275103569030762, 0.19308723509311676, 0.2495243400335312, -0.7101414203643799, -0.07499580085277557, -0.5979288220405579, -0.423862099647522, -0.35833466053009033, 0.06577394902706146, 0.5299686193466187, 0.4640010893344879, -0.9665706753730774, 0.0065386295318603516, 1.0564709901809692, -0.9828988909721375, -0.23279288411140442, -0.6185358762741089, -0.924900472164154, -0.46946224570274353, -0.7693082690238953, 0.3477574586868286, -0.4551454186439514, -1.3004581928253174, -0.5473873019218445, -0.4904305934906006, 0.10134521871805191, -0.7851577997207642, -0.1398417353630066, 0.8761336207389832, -0.12315058708190918, 0.8322192430496216, -0.39356720447540283, 0.11351998895406723, -0.7007197737693787, 0.7200241684913635, -0.2788134515285492, -0.24704404175281525, -0.6678914427757263, 0.20009173452854156, 0.9898447394371033, 0.7707079648971558, 0.7526775598526001, 0.5260705947875977, 0.5249796509742737, 0.6710962057113647, 0.046987444162368774, -0.21337898075580597, 0.04492022097110748, 0.2198365330696106, 0.30380189418792725, -0.3765871822834015, -0.11854751408100128, 0.3620243966579437, -0.8385960459709167, -0.32653316855430603, 0.713739275932312, 0.6545136570930481, 1.2750369310379028, 0.3916856348514557, 0.785868227481842, -0.19238032400608063, -0.1525554656982422, 0.020955558866262436, -0.49903085827827454, 0.9416297674179077, 0.5828056931495667, -0.0722537636756897, 0.5994181036949158, -1.1188567876815796, 0.10351072251796722, -0.7207244634628296, -0.4984204173088074, -0.8096540570259094, 0.18011435866355896, 0.9565514326095581, -0.15393893420696259, -0.9081364274024963, 0.9111977815628052, 1.0935875177383423, -0.3910906910896301, -0.7674028277397156, -0.3003276586532593, 0.22950905561447144, 0.48956984281539917, 0.43624240159988403, -0.3732295036315918, -0.2312670797109604, 0.013126324862241745, -0.34452563524246216, -0.3884854018688202, 0.8097610473632812, -0.3466103971004486, -0.4863629937171936, -1.2153193950653076, 0.44616684317588806, 0.4748398959636688, -0.7616560459136963, -0.10624860227108002, 0.1813899278640747, 0.028500637039542198, 0.019152285531163216, 0.9953367114067078, 0.0944482758641243, 1.0448013544082642, 0.5244965553283691, -0.6039860248565674, 0.7494000196456909, 1.4019547700881958, 0.2270280420780182, -0.5382144451141357, 0.7250616550445557, 0.37882766127586365, 0.12969160079956055, -0.5942366123199463, -0.07180619984865189, 0.06966392695903778, 0.3168462812900543, -0.4400121867656708, -0.8366104364395142, -0.21872909367084503, 2.273635149002075, 0.635701060295105, -0.6881745457649231, 0.6042446494102478, -0.07011879235506058, -0.07908092439174652, -0.4898541569709778, -0.4108656644821167, -0.7285102009773254, 0.7322966456413269, -0.26087555289268494, 0.11646150797605515, -0.37514930963516235, 0.5276262760162354, 0.6434467434883118, 0.3107970952987671, 1.07694411277771, 0.1968429982662201, -0.6798378229141235, 0.4157398045063019, -0.8171242475509644, 0.1432701200246811, -0.12088160216808319, -0.49819475412368774, 0.23451873660087585, -1.3125395774841309, 0.3253321945667267, 0.31344476342201233, -0.6642342209815979, 0.5588457584381104, 0.6691862344741821, -0.9133607745170593, 0.06850975006818771, -0.3948694169521332, -1.191189169883728, -0.22749076783657074, -0.2018890082836151, 0.4077981412410736, 1.2295258045196533, -0.06516718864440918, -0.029601914808154106, -1.0234524011611938, -0.8238799571990967, -0.058365147560834885, -0.0503610298037529, 0.04746090993285179, 0.3819830119609833, -0.25126275420188904, -0.6782487034797668, 0.26916536688804626, -1.0671441555023193, -0.8764973282814026, -0.20791040360927582, -0.6525862812995911, -1.1897250413894653, 0.31941989064216614, 0.603392481803894, -0.44752585887908936, 0.13575948774814606, -0.1329898089170456, -0.2629755139350891, -0.9610852003097534, 0.37693560123443604, -1.1491990089416504, -0.3705466687679291, -0.5210555195808411, -1.5667110681533813, -0.8276094794273376, -0.5881763696670532, -0.03459784761071205, 0.48798272013664246, -1.0512961149215698, -0.6378806829452515, -0.7676553726196289, 1.5053168535232544, -0.12419050931930542, -0.5378066897392273, 1.3219839334487915, 0.26627421379089355, 0.6976341009140015, 0.47981855273246765, 0.7861310243606567, 0.2661123275756836, -0.2338380515575409, 0.015867168083786964, 0.0013551730662584305, -0.0378306582570076, 1.3346132040023804, -0.08035647124052048, 0.4469560980796814, -0.002185460180044174, -0.17751222848892212, -0.13551387190818787, 0.051977332681417465, -0.886319100856781, -0.40313926339149475, 0.4063786268234253, -0.27102482318878174, 0.6758032441139221, -0.8657957315444946, -0.7341583371162415, -0.050888847559690475, -0.3934870958328247, -0.45259541273117065, 0.016656003892421722, 0.5815451145172119, -0.10052216053009033, -1.1516903638839722, 0.832090437412262, -0.5726155638694763, -0.08417540788650513, -1.602399468421936, -0.8060837984085083, 0.22210128605365753, 0.21664366126060486, 0.6899545192718506, -0.5209863185882568, 0.9422368407249451, 0.46065428853034973, 0.6300008296966553, 0.6646401882171631, -0.3648054897785187, -0.38844141364097595, 0.822163999080658, 0.8306391835212708, -0.9092362523078918, 0.3246057629585266, -0.43123045563697815, -0.7041870355606079, 1.3071298599243164, -0.366293340921402, 0.5541442036628723, 0.8113206624984741, -0.07291857898235321, -0.1323707401752472, -0.9278882741928101, -0.509712815284729, 0.4594009816646576, -0.5438634753227234, 0.0402398407459259, 0.17833197116851807, -0.7675198912620544, 0.28961119055747986, 0.4814383387565613, 0.9810733795166016, 0.24619585275650024, 0.45116347074508667, 0.11198007315397263, 0.030632805079221725, -0.9661092758178711, 0.0939900279045105, -0.33859360218048096, -0.46278440952301025, -0.5331777334213257, 0.3024255931377411, 0.7867016196250916, 1.5118449926376343, 1.5321561098098755, -0.21987107396125793, 0.8004183769226074, 1.0895298719406128, 1.1794852018356323, 0.1447429060935974, -0.27786052227020264, -0.17439967393875122, -0.17148059606552124, -0.7937490940093994, 0.06218211352825165, -1.0612996816635132, -0.134118914604187, -0.023471860215067863, -0.6959049701690674, -0.5473273992538452, -0.4064701795578003, 0.002896353602409363, -0.6807829141616821, -0.09238120913505554, 0.26466190814971924, 0.22154411673545837, 0.7110850811004639, -0.12211935967206955, -0.268065869808197, 0.38248011469841003, 0.6293134689331055, 0.0679243877530098, 0.13830150663852692, -0.5240658521652222, 1.219245195388794, -0.38758501410484314, 0.3336288034915924, -0.45287150144577026, -0.2536568343639374, 0.022776102647185326, -0.1163681298494339, 1.1858232021331787, -0.7754631638526917, -0.35377037525177, -0.8101058006286621, 1.0652310848236084, -0.7371259331703186, -0.39738908410072327, -1.0828492641448975, 1.444990634918213, 0.36333686113357544, -0.5950448513031006, -0.04707959294319153, -0.4712221324443817, 0.17247027158737183, -0.715339183807373, 0.03797708451747894, 1.0548977851867676, 0.3143754005432129, -0.7911570072174072, -0.44048288464546204, 1.0932673215866089, -0.18282850086688995, 0.13864780962467194, 0.1853216141462326, -0.13089017570018768, 1.1665658950805664, 0.3133753836154938, 0.699660062789917, 0.759121298789978, -0.8341694474220276, 0.08216738700866699, 0.5018881559371948, 1.0238654613494873, 0.045861825346946716, 0.13730092346668243, -0.09406701475381851, 1.6074745655059814, 0.01129714772105217, 0.14503249526023865, 1.1087737083435059, -0.6519383192062378, 0.4866137206554413, -0.24272532761096954, -0.5704152584075928, -0.4176703095436096, -0.8042948246002197, 0.17811673879623413, -0.8167643547058105, -0.41887202858924866, -0.19573217630386353, 1.0127867460250854, -0.8669375777244568, 0.0297623872756958, -0.756564199924469, 1.0621600151062012, 0.15557090938091278, -0.3950573205947876, 0.4814176559448242, -1.2014185190200806, 0.3746950924396515, -0.5729519128799438, -0.5657200217247009, -0.2777056396007538, -0.5889535546302795, 1.6377671957015991, -0.11628079414367676, 0.2047506421804428, -0.5987480282783508, -0.4639182984828949, -0.5080989003181458, 0.15958735346794128, -0.3460739850997925, -0.19687332212924957, 0.17925786972045898, -0.07531718909740448, 0.3190838694572449, -1.1137150526046753, 0.027028648182749748, 0.10706277936697006, -0.786180853843689, 1.0820149183273315, 0.0676846131682396, -0.5906199812889099, 0.7219001650810242, -0.9066491723060608, 1.1709039211273193, 0.889811635017395, -0.034078918397426605, 0.33509987592697144, -0.3930501639842987, -0.9930731654167175, -0.4727800488471985, 1.19788658618927, 0.39974042773246765, 0.3747997581958771, 1.869533658027649, 0.5353427529335022, 0.7052319645881653, 0.23293934762477875, -1.1192495822906494, 0.4041872024536133, -0.3588293790817261, -0.21030032634735107, -0.9293853640556335, -1.1494934558868408, 0.4425659477710724, 0.051006805151700974, 0.04104718565940857, 0.6634647250175476, 0.47568726539611816, 2.188441514968872, -0.21892884373664856, -0.0161183662712574, 0.1508212685585022, -0.7381160259246826, 0.4981902837753296, 0.2743193209171295, -0.47261857986450195, -0.49900174140930176, 0.6898798942565918, 0.5038102865219116, 0.34923508763313293, -0.1638583540916443, 0.15394975244998932, 0.43540722131729126, -0.30304569005966187, -0.9424941539764404, 0.011410290375351906, -1.092010259628296, 0.17171023786067963, -0.15554645657539368, -0.18984699249267578, -0.5912684202194214, -0.5105380415916443, 0.8195188641548157, -0.37337109446525574, 0.3058006763458252, -0.5972724556922913, 0.6969861388206482, 0.41494590044021606, 1.024333119392395, -0.24892064929008484, -0.6446018815040588, -0.0435771569609642, -0.22152984142303467, -0.012030958198010921, -0.7072649002075195, -0.1393945962190628, -0.6548402905464172, -1.0310088396072388, -0.7852085828781128, 1.4984668493270874, -0.26243463158607483, 0.4298070967197418, -0.9094841480255127, 0.052501752972602844, 0.8930417895317078, 1.7080692052841187, -0.6191803812980652, 0.6155276894569397, 0.019285092130303383, 0.31182152032852173, 0.2266562134027481, 0.4274917244911194, -0.8253052234649658, 0.3674454092979431, -0.6912070512771606, -0.04427909106016159, -0.18505056202411652, 0.8667874336242676, -0.21206234395503998, 0.7634763121604919, -0.21809343993663788, 0.235274076461792, 0.7831012010574341, 0.2932202219963074, 1.6685093641281128, 0.16268160939216614, 1.1632966995239258, 0.22296838462352753, 0.8300313353538513, 0.9158486723899841, 0.5726034045219421, -0.3832698166370392, -0.2561257481575012, 0.08156140893697739, 0.4466225206851959, 0.1544516235589981, 0.3354450464248657, 0.6569840312004089, -0.05895284563302994, 0.31050947308540344, 0.3941090404987335, -0.08907757699489594, 0.374954491853714, 0.4663066864013672, 0.3820387125015259, -0.040987271815538406, -0.6012201905250549, -0.4893884062767029, 0.4999065697193146, -0.9318278431892395, -0.6015385985374451, 0.07476461678743362, -0.5020935535430908, -0.6782195568084717, 0.683219313621521, 0.486665278673172, 0.2922327518463135, 0.3697029948234558, 0.03572787344455719, -0.08521200716495514, 0.36356663703918457, -0.4704844355583191, 0.8339800834655762, -0.17679418623447418, 0.55230712890625, 0.6818598508834839, 0.9447755217552185, 0.43726903200149536, 1.0239331722259521, 0.1683591902256012, -0.5529424548149109, 0.1910724937915802, 0.5893571972846985, -0.36923402547836304, 0.6082836389541626, -0.18567268550395966, 0.3736043870449066, 0.8520816564559937, -0.5474309921264648, 0.5192566514015198, -0.4788810610771179, -0.21132926642894745, 0.12813538312911987, 0.8497582077980042, 0.7805578112602234, -0.833868682384491, -0.31140202283859253, -0.07270720601081848, -0.454795241355896, 0.4720010459423065, -0.22759313881397247, -0.8376511335372925, -0.6364320516586304, 0.5480577945709229, -0.467008501291275, -0.035204652696847916, 1.5845122337341309, -0.4218668043613434, -0.2095354199409485, 0.3887205719947815, -0.8208023309707642, 0.33274734020233154, 0.9987019300460815, 0.5408844351768494, 0.12456350028514862, 1.124189853668213, -0.8487181067466736, -0.12929467856884003, 0.7679952383041382, 0.058899398893117905, 0.0779438316822052, 0.78458571434021, -0.09440557658672333, -0.13568434119224548, -0.006081763654947281, -0.18570037186145782, 0.005293948110193014, 0.2271084487438202, -0.8392688632011414, 0.09909024834632874, 0.10907971858978271, 0.06537670642137527, -0.01821119152009487, 0.390798956155777, 0.5407395362854004, -0.039750732481479645, -0.10784801840782166, -0.40497443079948425, -0.3379342555999756, 0.9925440549850464, -1.0298938751220703, -1.218425989151001, -0.37023818492889404, 0.45165467262268066, 0.07682419568300247, 0.44157418608665466, -0.397499144077301, -1.0791739225387573, 1.3557157516479492, 0.5493499636650085, -0.806052565574646, -0.6244906187057495, 0.16522838175296783, -1.1301088333129883, -0.445288747549057, 0.5989846587181091, 0.9037418961524963, 1.407365322113037, 0.266126424074173, -0.7504798173904419, -0.7552822828292847, -0.15925034880638123, 0.4085763990879059, 0.4981786608695984, -0.017093215137720108, -0.28162431716918945, -0.9051836729049683, 1.1940789222717285, -0.29602470993995667, -0.08827870339155197, -0.762412965297699, 0.3285483419895172, 0.13840213418006897, -0.13394613564014435, 0.5985749363899231, 0.2454182207584381, -0.516996443271637, -0.5675912499427795, -0.06059342995285988, 0.44863417744636536, 0.244121253490448, -0.6903651356697083, -0.41876256465911865, 0.06660375744104385, 0.35941585898399353, -0.8821203708648682, -1.1803172826766968, -0.3216383457183838, -0.5331739187240601, -0.5727825164794922, -0.29154032468795776, -0.40957337617874146],
            ]
array_face_saved = np.array(face_saved) 
array_face_saved

array([[ 0.32659087,  0.96631968,  0.19752678, ..., -0.57779914,
        -0.29330075, -0.40724397],
       [ 0.29449713,  0.96432608,  0.24630725, ..., -0.57278252,
        -0.29154032, -0.40957338]])

In [11]:
try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        
        # Align the depth frame to color frame
        aligned_frames = align.process(frames)
        
        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        
        if not aligned_depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        # dep_img = depth_image
        # rgb_img = color_image
        
        dets = detector(color_image, 1)
        for d in dets:
            x, y, w, h = d.left()-5, d.top()-15, d.right()+5, d.bottom()+10
            xy = x, y
            wh = w, h

            rgb_img = color_image[y+2:h-2, x+2:w-2]
#             rgb_img = color_image[y+15:h-10, x+5:w-5]
            rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
            
            dep_img = depth_image[y+2:h-2, x+2:w-2]
#             dep_img = depth_image[y+15:h-10, x+5:w-5]
            dep_img = np.expand_dims(dep_img, axis=-1)
            img = np.concatenate((rgb_img, dep_img), axis=-1)
            # print(rgb_img)
            # print(dep_img)
            # print(img)
            
            name = "unknown"
            outputs = predict(img)
#             acc = max(outputs[0]).item()
            array_output = np.array(outputs[0].tolist())
#             array_output = outputs[0].tolist()
#             print(array_output)
            min_dist = 100 
            for i,face_saved in enumerate(array_face_saved):
                dist = np.linalg.norm(face_saved - array_output) 
                if dist < min_dist:
                    min_dist = dist
                    index = i
            print(min_dist)
            if min_dist < 1.5:
                name = list_name[index]
                
#             print(acc)
#             if(acc<0.6):
#                 # print(outputs)
#                 name = "unknown"
#             else:
#                 _, preds = torch.max(outputs, 1)
#                 n_name = preds.item()
#                 name = str(list_name[n_name])

            cv2.rectangle(color_image, xy, wh, (255,0,0), 2)
            cv2.putText(color_image, name, (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)
        
        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', color_image)
        key = cv2.waitKey(10)
        
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break

finally:
    # Stop streaming
    pipeline.stop()

6.111930770567182
0.795926403003634
0.8036278809871665
0.7978603603892443
1.4952595779910647
2.7123130614951716
0.1197623433906746
0.11873281564425801
0.13153922787731587
0.12856024752275091
0.15930777859869555
0.14769803868994594
0.1560185948724571
0.1430700865999103
0.14641079906395912
2.8303857693806767
0.27057558434760876
0.13612616752534645
0.17416604807570715
0.2718313606874246
0.25499579254919486
0.2687346634107885
0.149934319234456
0.14459760703108052
0.1726566789585981
0.3457216551774375
0.28633092372098506
0.10671150294755352
0.11109832291878886
0.33613413976411366
0.1533641133641142
0.1430470981695332
0.14770396108187317
0.15193370779191834
0.15733154228032162
0.16393603138246843
0.2915230245996792
2.6658740501254057
2.6705222947980687
0.29154583661534716
2.65977653411405
0.20093109963419445
0.19755281276376055
0.17305793370230463
0.17280529937022276
0.17090046105323622
2.6704092039084197
2.6870691779833886
2.6910432621666276
2.6846257633517254
2.680159068399894
0.2400248220